# Wrangling

In [1]:
import pandas as pd
import re
import sys
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
import datetime
from datetime import datetime
import calendar
from sklearn.metrics import jaccard_score as jscore
from scipy.spatial.distance import pdist, squareform
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from nltk.corpus import stopwords as sw
import chart_studio.plotly as py
import cufflinks as cf
import plotly.express as px
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
init_notebook_mode(connected=True)
cf.go_offline()
%matplotlib inline

In [2]:
games = pd.read_csv('metacritic_games_master2.csv').drop(['Unnamed: 0'], axis = 1)
genre_norepeat = [', '.join(sorted(set(i.split(',')))).strip() for i in games.genre.values]
games['genres'] = genre_norepeat
games = games.drop(columns='genre')
games.head()

,title,release_date,platforms,developer,esrb_rating,ESRBs,metascore,userscore,critic_reviews,user_reviews,num_players,summary,genres
0,Burnout 3: Takedown,"Sep 7, 2004",Xbox,Criterion Games,T,Mild Language Mild Violence,94,7.7,76,191.0,"1-2 Players, 8 Players Online",Burnout 3 challenges you to crash into (and th...,"Arcade, Racing, Driving"
1,Jet Grind Radio,"Oct 30, 2000",Dreamcast,Smilebit,T,Animated Violence Mild Language,94,8.3,24,105.0,1 Player,"Join a graffiti crew, stamp your territory and...","3D, Platformer, Action"
2,Metal Gear Solid 4: Guns of the Patriots,"Jun 12, 2008",PlayStation 3,Kojima Productions,M,Blood Crude Humor Strong Language Suggestive T...,94,8.8,82,4231.0,"1 Player, 16 Players Online Up to 16 Players",Metal Gear Solid 4: Guns of the Patriots featu...,"General, Modern, Action Adventure"
3,Tom Clancy's Splinter Cell Chaos Theory,"Mar 28, 2005",Xbox,Ubisoft Montreal,M,Blood Strong Language Violence,94,9.1,70,233.0,"1-4 Players, 4 Players Online","As Sam Fisher, Third Echelon's most skilled Sp...","Modern, Action Adventure"
4,Call of Duty: Modern Warfare 2,"Nov 10, 2009",Xbox 360,Infinity Ward,M,Blood Drug Reference Intense Violence Language,94,6.6,100,3055.0,Up to 18 Players,Modern Warfare 2 continues the gripping and he...,"Arcade, First-Person, Modern, Shooter, Action"


In [3]:
games.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 19317 entries, 0 to 19316
Data columns (total 13 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   title           19317 non-null  object 
 1   release_date    19317 non-null  object 
 2   platforms       19317 non-null  object 
 3   developer       19298 non-null  object 
 4   esrb_rating     17202 non-null  object 
 5   ESRBs           7855 non-null   object 
 6   metascore       19317 non-null  int64  
 7   userscore       19317 non-null  object 
 8   critic_reviews  19317 non-null  int64  
 9   user_reviews    17953 non-null  float64
 10  num_players     19304 non-null  object 
 11  summary         19199 non-null  object 
 12  genres          19317 non-null  object 
dtypes: float64(1), int64(2), object(10)
memory usage: 1.9+ MB


In [4]:
# try to convert release date column to datetime type
try:
    pd.to_datetime(games.release_date, infer_datetime_format=True)
except Exception as e:
    print('\n', sys.exc_info()[0],'\n', sys.exc_info()[1])


 <class 'dateutil.parser._parser.ParserError'> 
 Unknown string format: 
PC,                                                                            PlayStation 4,                                                                            PlayStation 5,                                                                            PlayStation Vita,                                                                            Xbox One 


In [5]:
# This summary was found in the release date column for some reason
summary = 'Welcome to the emergent open world, Wild West-themed action MMO featuring world exploration, resource gathering, PvP combat, PvE missions and much, much more.\r\n'
games[games.release_date==summary]

,title,release_date,platforms,developer,esrb_rating,ESRBs,metascore,userscore,critic_reviews,user_reviews,num_players,summary,genres
19240,Wild West Online,"Welcome to the emergent open world, Wild West-...",PC,NaN,NaN,NaN,29,1.5,8,52.0,Massively Multiplayer Online Multiplayer,NaN,"Massively Multiplayer, Role-Playing"


In [6]:
# put summary in summary column and get release date from website for Wild West Online
games.iloc[19240, -2] = summary.strip()
wwo_rdate = 'May 10, 2018'
games.iloc[19240, 1] = wwo_rdate
games.iloc[19240, :]

title                                              Wild West Online
release_date                                           May 10, 2018
platforms                                                        PC
developer                                                       NaN
esrb_rating                                                     NaN
ESRBs                                                           NaN
metascore                                                        29
userscore                                                       1.5
critic_reviews                                                    8
user_reviews                                                   52.0
num_players                Massively Multiplayer Online Multiplayer
summary           Welcome to the emergent open world, Wild West-...
genres                          Massively Multiplayer, Role-Playing
Name: 19240, dtype: object

Lets check for more rows without dates in the `release_date` column

In [7]:
# look for any instances of values that cannot be converted to datetime
non_date_indices = []
for i in range(len(games.release_date)):
    try:
        datetime.strptime(games.release_date[i].replace(',', ''), '%b %d %Y')
    except:
        non_date_indices.append(i)
non_date_indices

[4744, 6392, 6954, 8882, 11501, 11764, 16180, 17071, 17343, 18193]

In [8]:
games.iloc[non_date_indices, :]

,title,release_date,platforms,developer,esrb_rating,ESRBs,metascore,userscore,critic_reviews,user_reviews,num_players,summary,genres
4744,ScourgeBringer,"\nPC, ...",Switch,Flying Oak Games,NaN,NaN,80,7.4,9,8.0,1 Player,NaN,"2D, Platformer, Action"
6392,Everyday Shooter,\nPlayStation 3,PC,Queasy Games,NaN,NaN,77,7.6,7,11.0,1 Player,NaN,"Shoot-'Em-Up, Shooter, Static, Top-Down, Ac..."
6954,Secret World Legends,Fight a secret war of the supernatural. Where ...,PC,Funcom,NaN,NaN,76,5.2,9,49.0,1 Player,NaN,"Action Adventure, General, Massively Multipl..."
8882,Orcs Must Die! 3,"\nPC, ...",Stadia,Robot Entertainment,T,NaN,73,7.7,15,81.0,1 Player,NaN,"Defense, Real-Time, Strategy"
11501,Tracks: The Train Set Game,"\nPC, ...",Xbox One,Whoop Group,E,NaN,69,5.5,8,11.0,1 Player,NaN,"Business / Tycoon, Management, Strategy"
11764,Buzz! Junior: Dino Den,The next installment in the Buzz series.,PlayStation 2,SCEE,NaN,NaN,69,tbd,8,NaN,1-4 Players,NaN,"Trivia / Game Show, Miscellaneous"
16180,Warcraft III: Reforged,"A Classic Favorite, Reforged. Warcraft III: Re...",PC,Blizzard Entertainment,T,NaN,59,0.6,46,30913.0,Online Multiplayer,NaN,"General, Real-Time, Strategy"
17071,Empyre: Lords of the Sea Gates,New York in 1911 is flooded. Avenues are now c...,PC,Coin Operated Games,NaN,NaN,56,tbd,10,NaN,1 Player,NaN,"Action RPG, Role-Playing"
17343,Road to Guangdong,"\nPC, ...",Xbox One,Just Add Oil Games,E,NaN,54,tbd,7,NaN,1 Player,NaN,"Civilian, Vehicle, Simulation"
18193,PixelJunk Raiders,PixelJunk Raiders is the latest creation from ...,Stadia,Q-Games,T,NaN,49,6.0,9,71.0,1 Player,NaN,"General, Action"


In [9]:
# Go through each game on the website since there are only 10 cases of this
def change_sum_and_date(idx, summary, rdate):   
    summ = summary
    games.iloc[idx, -2] = summary.strip()
    date = rdate
    games.iloc[idx, 1] = date

In [10]:
# ScourgeBringer, Switch--> https://www.metacritic.com/game/switch/scourgebringer/details
idx = non_date_indices[0]
summary = 'ScourgeBringer is a fast-paced free-moving roguelite platformer. Help Kyhra to explore the unknown and slash her way through ancient machines guarding the seal of her past, and maybe the redemption of humanity.'
rdate = 'Oct 21, 2020'
change_sum_and_date(idx, summary, rdate)

In [11]:
# Everyday Shooter, PC--> https://www.metacritic.com/game/pc/everyday-shooter/details
idx = non_date_indices[1]
summary = "Everyday Shooter is an album of games exploring the expressive power of abstract shooters. Dissolute sounds of destruction are replaced with guitar riffs harmonizing over an all-guitar soundtrack, while modulating shapes celebrate the flowing beauty of geometry. [Queasy Games]"
rdate = "Mar 29, 2006"
change_sum_and_date(idx, summary, rdate)

In [12]:
# Secret World Legends, PC--> https://www.metacritic.com/game/pc/secret-world-legends/details
idx = non_date_indices[2]
summary = "Fight a secret war of the supernatural. Where will you stand in the secret war between good and evil as supernatural forces threaten the modern-day world? Use a wide range of weapons and superhuman abilities to destroy the sinister evils that are threatening humanity’s existence."
rdate = "Jun 26, 2017"
change_sum_and_date(idx, summary, rdate)

In [13]:
# Orcs Must Die! 3, Stadia--> https://www.metacritic.com/game/stadia/orcs-must-die!-3/details
idx = non_date_indices[3]
summary = "Orcs Must Die! 3 ushers orc-slaying mayhem to a previously unimaginable scale. All new War Scenarios pit players and their friends against the largest orc armies ever assembled. Mountable War Machines give players the essential firepower to heave, stab, carbonize, and disarticulate the abominable intruders. Orcs Must Die! 3 is the long-awaited successor to the award-winning series."
rdate = "Jul 14, 2020"
change_sum_and_date(idx, summary, rdate)

In [14]:
# Tracks: The Train Set Game, Xbox One--> https://www.metacritic.com/game/xbox-one/tracks-the-train-set-game/details
idx = non_date_indices[4]
summary = "Create colorful railway systems, decorate beautiful towns, and even ride in your train."
rdate = "Nov 14, 2019"
change_sum_and_date(idx, summary, rdate)

In [15]:
# Buzz! Junior: Dino Den, PlayStation 2--> https://www.metacritic.com/game/playstation-2/buzz!-junior-dino-den/details
idx = non_date_indices[5]
summary = "The next installment in the Buzz series."
rdate = "Sep 25, 2008"
change_sum_and_date(idx, summary, rdate)

In [16]:
# Warcraft III: Reforged, PC--> https://www.metacritic.com/game/pc/warcraft-iii-reforged/details
idx = non_date_indices[6]
summary =\
"""
A Classic Favorite, Reforged. Warcraft III: Reforged is a reimagining of the real-time strategy game that laid the foundation for Azeroth's most epic stories. It is a remake featuring a thorough visual overhaul, a suite of contemporary social and matchmaking features, and more. Command the Night Elves, Undead, Orcs, and Humans as alliances shift and armies clash in this timeless real-time strategy game. AZEROTH, BORN ANEW! Experience an Epic Story. The founding of Orgrimmar. The fall of Lordaeron. The reign of the Burning Legion. The rise of the Lich King. Veterans and newcomers alike will experience these pivotal events in Azeroth's history like never before. The Battlefield Awaits. Play through the points of view of four distinct factions: the mighty Orcs, the noble Humans, the ancient Night Elves, and the insidious Undead. Gather resources, build your base, and assemble an army. Recruit powerful heroes to lead your forces, and meet your foes in battle. Destroy the enemy base to win the game! Vast Campaigns. Relive the events of Warcraft III: Reign of Chaos and The Frozen Throne. Enjoy an epic saga retold with over four hours of reshot cutscenes and updated voice-overs that breathe new life into the earliest renditions of Azeroth's key characters, across more than 60 single-player missions that span Kalimdor, Northrend, Lordaeron, and beyond. Reimagined Visuals. Every character, structure, and environment has been recreated to emphasis the depth, dimension, and personality of this rough-hewn fantasy world. Behold Warcraft III's classic units in glorious 4K, with updated animations that bring them to life like you've never seen. World Editor & Custom Maps. Return to the game that started it all. Discover a vast universe of player-created games, including tower defense, MOBAs, RPGs, survival games, and moreor build your own with the upgraded World Editor. Multiplayer Matchmaking. Challenge opponents with modern multiplayer matchmaking, search for custom game lobbies, and connect with friends through the Blizzard Battle.net app. With updated UI and countless quality-of-life improvements, its never been easier to get into Warcraft III."
"""
rdate = "Jan 28, 2020"
change_sum_and_date(idx, summary, rdate)

In [17]:
# Empyre: Lords of the Sea Gates, PC--> https://www.metacritic.com/game/pc/empyre-lords-of-the-sea-gates/details
idx = non_date_indices[7]
summary = games.iloc[idx].release_date.strip()
rdate = "Oct  4, 2017"
change_sum_and_date(idx, summary, rdate)

In [18]:
# Road to Guangdong, Xbox One--> https://www.metacritic.com/game/xbox-one/road-to-guangdong/details
idx = non_date_indices[8]
summary = "A family emergency reunites a young art graduate and her elderly aunt for a memorable road trip across 1990s China. Sunny’s life is at a crossroads. She knows everything’s about to change. She just doesn’t know how yet. She glances again at the contract. The good news is she’s now the proud owner of her family‘s cherished restaurant. The bad news? It’s seen better days. A tap at the door announces the arrival of her aunt, Guu Ma. Yep, everything’s about to change alright. But the “how” part will be decided by a bunch of strangers she’s only seen in faded photographs."
rdate = "Aug 28, 2020"
change_sum_and_date(idx, summary, rdate)

In [19]:
# PixelJunk Raiders, Stadia--> https://www.metacritic.com/game/stadia/pixeljunk-raiders/details
idx = non_date_indices[9]
summary = games.iloc[idx].release_date.strip()
rdate = "Mar  1, 2021"
change_sum_and_date(idx, summary, rdate)

In [20]:
# try to covnvert release_date column again
try:
    games = games.astype({'release_date': 'datetime64[ns]'})
    print('Converted!')
except Exception as e:
    print('\n', sys.exc_info()[0],'\n', sys.exc_info()[1])

Converted!


In [21]:
games.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 19317 entries, 0 to 19316
Data columns (total 13 columns):
 #   Column          Non-Null Count  Dtype         
---  ------          --------------  -----         
 0   title           19317 non-null  object        
 1   release_date    19317 non-null  datetime64[ns]
 2   platforms       19317 non-null  object        
 3   developer       19298 non-null  object        
 4   esrb_rating     17202 non-null  object        
 5   ESRBs           7855 non-null   object        
 6   metascore       19317 non-null  int64         
 7   userscore       19317 non-null  object        
 8   critic_reviews  19317 non-null  int64         
 9   user_reviews    17953 non-null  float64       
 10  num_players     19304 non-null  object        
 11  summary         19210 non-null  object        
 12  genres          19317 non-null  object        
dtypes: datetime64[ns](1), float64(1), int64(2), object(9)
memory usage: 1.9+ MB


The userscore column is of type object. It should be of type float64. Looking at previous outputs for the release_dates we can see that there are 'tbd's in the userscore column. Lets see if we can fill these in with anything meaningful.

In [22]:
# number of rows with tbd as userscore
len(games[games.userscore=='tbd'].title)

1364

In [23]:
no_userscore_titles = list(games[games.userscore=='tbd'].title)

In [24]:
reviews = pd.read_csv('metacritic_reviews.csv').drop(['Unnamed: 0'],axis=1)
reviews.head()

,ids,game,rating,review
0,0,Rockstar Games presents Table Tennis,9,Amazingly Addicting!
1,0,Disney's Magical Mirror Starring Mickey Mouse,10,this game lets you able to play as mickey mou...
2,1,Deus Ex: Invisible War,1,Yet another disappointment to add to 2003's r...
3,2,Mobile Suit Gundam: Journey to Jaburo,10,I love this Game!
4,3,Dead to Rights: Reckoning,10,This is a really good game for people ...


In [25]:
# review rows of games that have no userscore in the game data
g = reviews[reviews.game.isin(no_userscore_titles)].game

In [26]:
# Attempt at dropping consecutive duplicates. To distinguis between platforms.   
g.loc[g.shift(-1)!=g].value_counts()

LEGO Marvel Super Heroes    151
Madden NFL 06               141
The Long Dark               135
Jurassic World Evolution    132
DiRT 2                      115
                           ... 
Jet Lancer                    1
Championship Surfer           1
Guitar Rock Tour              1
Shrek: Reekin' Havoc          1
Crazy Machines Elements       1
Name: game, Length: 798, dtype: int64

Update: (5/11) I realized this won't work as the data was not scraped consecutively but in paralell, so not all reviews from the same game are not necessarily, and almost certainly not consecutive So there is no way of seperating same titles from different platforms. For example, if this method was going to work, "Lego Marvel Super Heroes" would only have 7 instances (the number of platforms) of  as opposed to 151. 

From now I will only be wrangling data if necessary for EDA or Modeling. 

In [27]:
# games.to_csv('games_eda.csv')